In [104]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import  cosine_similarity

#set the max columns to none
pd.set_option('display.max_columns', None)

### I will create the Cosine similarity matrix from word to vec embeddings and save it in order to quick access

In [106]:
# Ready embeddings

embeddings_path = f'./files/parquet/Ekstra_Bladet_word2vec/document_vector.parquet'
embeddings_df = pd.read_parquet(embeddings_path)
print(f'embeddings df shape:          ',embeddings_df.shape)

# Ready embeddings
size = 'demo'
final_lst_path =  f'./files/parquet/ebnerd_{size}/final_lst.pkl'
final_lst_df = pd.read_pickle(final_lst_path)
print(f'embeddings df shape:          ',final_lst_df.shape)

final_lst = final_lst_df['articles_id'].tolist()
print('final article ids list:         ',len(final_lst))

# Raw data
size = 'demo'
articles_path = f'./files/parquet/ebnerd_{size}/articles.parquet'
articles_df = pd.read_parquet(articles_path)
print('Articles df shape:           ',articles_df.shape)

embeddings df shape:           (125541, 2)
embeddings df shape:           (5638, 1)
final article ids list:          5638
Articles df shape:            (11777, 21)


In [107]:
print(embeddings_df.index.min())
print(embeddings_df.index.max())

print(embeddings_df.tail(70000).index.min())
print(embeddings_df.tail(70000).index.max())

0
125540
55541
125540


In [108]:
embeddings_df.set_index('article_id', inplace=True)
embeddings_df = embeddings_df.loc[final_lst, :]
embeddings_df.head(5)

,document_vector
article_id,
9306121,"[0.044088427, -0.012972383, 0.021840084, 0.063..."
9568274,"[0.015295008, -0.056765914, 0.086636186, 0.040..."
9764895,"[0.07058594, -0.02757115, -0.038418178, 0.0118..."
5996581,"[0.010439406, -0.02596113, 0.045929633, 0.0262..."
9666599,"[0.023732126, 0.029601572, -0.038265876, 0.011..."


In [92]:
articles_df.head()

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3037230,Ishockey-spiller: Jeg troede jeg skulle dø,ISHOCKEY: Ishockey-spilleren Sebastian Harts h...,2023-06-29 06:20:57,False,Ambitionerne om at komme til USA og spille ish...,2003-08-28 08:55:00,None,article_default,https://ekstrabladet.dk/sport/anden_sport/isho...,[],[],"[Kriminalitet, Kendt, Sport, Katastrofe, Mindr...",142,"[327, 334]",sport,NaN,NaN,NaN,0.9752,Negative
1,3044020,Prins Harry tvunget til dna-test,Hoffet tvang Prins Harry til at tage dna-test ...,2023-06-29 06:21:16,False,Den britiske tabloidavis The Sun fortsætter me...,2005-06-29 08:47:00,"[3097307, 3097197, 3104927]",article_default,https://ekstrabladet.dk/underholdning/udlandke...,"[Harry, James Hewitt]","[PER, PER]","[Kriminalitet, Kendt, Underholdning, Personfar...",414,[432],underholdning,NaN,NaN,NaN,0.7084,Negative
2,3057622,Rådden kørsel på blå plader,Kan ikke straffes: Udenlandske diplomater i Da...,2023-06-29 06:21:24,False,Slingrende spritkørsel. Grove overtrædelser af...,2005-10-10 07:20:00,[3047102],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,[],[],"[Kriminalitet, Transportmiddel, Bil]",118,[133],nyheder,NaN,NaN,NaN,0.9236,Negative
3,3073151,Mærsk-arvinger i livsfare,FANGET I FLODBØLGEN: Skibsrederens oldebørn må...,2023-06-29 06:21:38,False,To oldebørn af skibsreder Mærsk McKinney Mølle...,2005-01-04 06:59:00,"[3067474, 3067478, 3153705]",article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,[],[],"[Erhverv, Privat virksomhed, Livsstil, Familie...",118,[133],nyheder,NaN,NaN,NaN,0.9945,Negative
4,3193383,Skød svigersøn gennem babydyne,44-årig kvinde tiltalt for drab på ekssvigersø...,2023-06-29 06:22:57,False,En 44-årig mormor blev i dag fremstillet i et ...,2003-09-15 15:30:00,None,article_default,https://ekstrabladet.dk/krimi/article3193383.ece,[],[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9966,Negative


In [109]:
# Unpack th lists in the column
def unpack_lists(df, column_name):
    # Create new columns by unpacking lists in the specified column
    unpacked_cols = pd.DataFrame(df[column_name].tolist(), index=df.index)
    # Rename the columns with a prefix based on the original column name
    unpacked_cols = unpacked_cols.add_prefix(column_name + '_')
    # Concatenate the unpacked columns back to the original DataFrame
    df = pd.concat([df, unpacked_cols], axis=1)
    return df

# Specify which columns to process
columns_to_process = ['document_vector']

# Unpack lists in each specified column
for col in columns_to_process:
    embeddings_df = unpack_lists(embeddings_df, col)

embeddings_df.drop('document_vector', axis=1, inplace=True)

In [110]:
embeddings_df.head()

,document_vector_0,document_vector_1,document_vector_2,document_vector_3,document_vector_4,document_vector_5,document_vector_6,document_vector_7,document_vector_8,document_vector_9,document_vector_10,document_vector_11,document_vector_12,document_vector_13,document_vector_14,document_vector_15,document_vector_16,document_vector_17,document_vector_18,document_vector_19,document_vector_20,document_vector_21,document_vector_22,document_vector_23,document_vector_24,document_vector_25,document_vector_26,document_vector_27,document_vector_28,document_vector_29,document_vector_30,document_vector_31,document_vector_32,document_vector_33,document_vector_34,document_vector_35,document_vector_36,document_vector_37,document_vector_38,document_vector_39,document_vector_40,document_vector_41,document_vector_42,document_vector_43,document_vector_44,document_vector_45,document_vector_46,document_vector_47,document_vector_48,document_vector_49,document_vector_50,document_vector_51,document_vector_52,document_vector_53,document_vector_54,document_vector_55,document_vector_56,document_vector_57,document_vector_58,document_vector_59,document_vector_60,document_vector_61,document_vector_62,document_vector_63,document_vector_64,document_vector_65,document_vector_66,document_vector_67,document_vector_68,document_vector_69,document_vector_70,document_vector_71,document_vector_72,document_vector_73,document_vector_74,document_vector_75,document_vector_76,document_vector_77,document_vector_78,document_vector_79,document_vector_80,document_vector_81,document_vector_82,document_vector_83,document_vector_84,document_vector_85,document_vector_86,document_vector_87,document_vector_88,document_vector_89,document_vector_90,document_vector_91,document_vector_92,document_vector_93,document_vector_94,document_vector_95,document_vector_96,document_vector_97,document_vector_98,document_vector_99,document_vector_100,document_vector_101,document_vector_102,document_vector_103,document_vector_104,document_vector_105,document_vector_106,document_vector_107,document_vector_108,document_vector_109,document_vector_110,document_vector_111,document_vector_112,document_vector_113,document_vector_114,document_vector_115,document_vector_116,document_vector_117,document_vector_118,document_vector_119,document_vector_120,document_vector_121,document_vector_122,document_vector_123,document_vector_124,document_vector_125,document_vector_126,document_vector_127,document_vector_128,document_vector_129,document_vector_130,document_vector_131,document_vector_132,document_vector_133,document_vector_134,document_vector_135,document_vector_136,document_vector_137,document_vector_138,document_vector_139,document_vector_140,document_vector_141,document_vector_142,document_vector_143,document_vector_144,document_vector_145,document_vector_146,document_vector_147,document_vector_148,document_vector_149,document_vector_150,document_vector_151,document_vector_152,document_vector_153,document_vector_154,document_vector_155,document_vector_156,document_vector_157,document_vector_158,document_vector_159,document_vector_160,document_vector_161,document_vector_162,document_vector_163,document_vector_164,document_vector_165,document_vector_166,document_vector_167,document_vector_168,document_vector_169,document_vector_170,document_vector_171,document_vector_172,document_vector_173,document_vector_174,document_vector_175,document_vector_176,document_vector_177,document_vector_178,document_vector_179,document_vector_180,document_vector_181,document_vector_182,document_vector_183,document_vector_184,document_vector_185,document_vector_186,document_vector_187,document_vector_188,document_vector_189,document_vector_190,document_vector_191,document_vector_192,document_vector_193,document_vector_194,document_vector_195,document_vector_196,document_vector_197,document_vector_198,document_vector_199,document_vector_200,document_vector_201,document_vector_202,document_vector_203,document_vector_204,document_

In [111]:
# Compute cosine similarity matrix
cosine_similarity_matrix = cosine_similarity(embeddings_df.values)

In [112]:
print(cosine_similarity_matrix.shape)
cosine_similarity_df = pd.DataFrame(cosine_similarity_matrix, index=embeddings_df.index, columns=embeddings_df.index)

(5638, 5638)


In [113]:
cosine_similarity_df.head()

article_id   9306121   9568274   9764895   5996581   9666599   9764909  \
article_id                                                               
9306121     1.000000  0.674356  0.747147  0.858023  0.876015  0.705622   
9568274     0.674356  1.000000  0.631668  0.671629  0.609433  0.610673   
9764895     0.747147  0.631668  1.000000  0.724137  0.729785  0.809719   
5996581     0.858023  0.671629  0.724137  1.000000  0.820750  0.657701   
9666599     0.876015  0.609433  0.729785  0.820750  1.000000  0.681974   

article_id   9764916   9732155   9699394   9666638   9470037   9699417  \
article_id                                                               
9306121     0.744126  0.932712  0.840868  0.821475  0.756688  0.787001   
9568274     0.595214  0.675617  0.600703  0.650559  0.706415  0.627728   
9764895     0.811588  0.739873  0.698493  0.676368  0.631210  0.689955   
5996581     0.712269  0.877603  0.813125  0.860892  0.722785  0.791969   
9666599     0.715353  0.893717  0.905076  0.763104  0.677125  0.796829   

article_id   9175145   9764978   9764986   9666682   9764988   9764990  \
article_id                                                               
9306121     0.938143  0.833373  0.847716  0.844094  0.900447  0.748510   
9568274     0.617262  0.696634  0.668160  0.712884  0.701985  0.648845   
9764895     0.708285  0.908992  0.918559  0.687887  0.763405  0.774689   
5996581     0.794762  0.792329  0.818323  0.788880  0.855028  0.737405   
9666599     0.855912  0.795754  0.825650  0.739944  0.856416  0.740282   

article_id   9470078   9764994   9371779   7045260   9142414   9765010  \
article_id                                                               
9306121     0.896509  0.839836  0.902916  0.781044  0.888400  0.766714   
9568274     0.609814  0.615809  0.651114  0.615929  0.605165  0.657503   
9764895     0.698342  0.757463  0.811084  0.651154  0.689522  0.908370   
5996581     0.867998  0.831835  0.865706  0.764914  0.794211  0.737558   
9666599     0.876400  0.878221  0.882968  0.749711  0.857646  0.742058   

article_id   9699487   9699499   9470132   8716485   9765067   9371867  \
article_id                                                               
9306121     0.921703  0.745840  0.922294  0.715127  0.887296  0.893337   
9568274     0.672864  0.571934  0.667695  0.802899  0.629249  0.634937   
9764895     0.731198  0.592512  0.735219  0.662993  0.748443  0.807853   
5996581     0.856783  0.766651  0.836405  0.731143  0.885064  0.838612   
9666599     0.868760  0.739106  0.868849  0.652410  0.896033  0.887287   

article_id   9765084   9699562   9699564   9732343   9699601   9765143  \
article_id                                                               
9306121     0.584318  0.941116  0.905542  0.839834  0.883929  0.774918   
9568274     0.450230  0.704522  0.629509  0.671759  0.608331  0.696306   
9764895     0.737019  0.747988  0.694266  0.760688  0.721447  0.722923   
5996581     0.526396  0.837560  0.851233  0.776681  0.772382  0.819976   
9666599     0.598749  0.858430  0.859451  0.782322  0.825280  0.778786   

article_id   9666842   9699615   9765153   9765156   9142564   9765159  \
article_id                                                               
9306121     0.954459  0.701722  0.836818  0.792253  0.913176  0.853325   
9568274     0.665570  0.533569  0.703264  0.587701  0.716451  0.646536   
9764895     0.734754  0.603593  0.745434  0.662502  0.759761  0.895547   
5996581     0.844295  0.745008  0.829551  0.714205  0.864504  0.790594   
9666599     0.861263  0.796107  0.797608  0.703895  0.853120  0.818138   

article_id   9765169   9765172   9142581   9765175   9765181   9699647  \
article_id                                                               
9306121     0.869914  0.838905  0.952242  0.762369  0.901172  0.896746   
9568274     0.682596  0.650013  0.674599  0.606422  0.674631  0.645170   
9764895     0.909140  0.886374  0.725400  0.614280  0.814875  0.697209 

In [98]:
# cosine_similarity_df = pd.merge(cosine_similarity_df, articles_df[['article_id', 'category_str']], left_on=cosine_similarity_df.index, right_on='article_id', how="left")

In [101]:
# cosine_similarity_df.head()

In [102]:
# cosine_similarity_df.set_index('article_id', inplace=True)

In [114]:
# Save the cosine similarity matrix to pickle file cause is very big
cosine_similarity_df_file_path = f'./files/pickle/cosine_similarity.pkl'

cosine_similarity_df.to_pickle(cosine_similarity_df_file_path)

In [5]:
# def create_pca_embeddings(df, n_components):
    
#     # Initialize PCA model
#     pca = PCA(n_components=n_components)
    
#     # Fit PCA on the numerical data
#     embeddings = pca.fit_transform(df.values)
    
#     # Create a new DataFrame to store PCA embeddings with the same index as the original DataFrame
#     embeddings_df = pd.DataFrame(embeddings, index=df.index)
    
#     # Rename columns of the embeddings DataFrame
#     embeddings_df.columns = [f'pca_embedding_{i}' for i in range(n_components)]
    
#     return embeddings_df


# # Specify columns to apply embeddings
# #columns_for_embeddings = list(df_all_articles_embeddings.columns)

# # Apply PCA embeddings to ID columns
# df_all_articles_embeddings = create_pca_embeddings(df_all_articles_embeddings, n_components=50)

In [45]:
# num_of_rows = 60000
# chunk_size = 200
# mult_way = 'full'

In [46]:
# ------------ There is a possibility that this way with chuncks doesn't create good embeddings -------

# # Function to compute cosine similarity in chunks avoiding memory issues lose some information
# def cosine_similarity_chunked(data, chunk_size):
#     n = data.shape[0]
#     similarity_matrix = np.zeros((n, n))
    
#     for start in tqdm(range(0, n, chunk_size)):
#         end = min(start + chunk_size, n)
#         chunk_sim = cosine_similarity(data[start:end], data)
#         similarity_matrix[start:end] = chunk_sim
        
#     return similarity_matrix



# # Convert DataFrame to numpy array
# cosine_similarity_matrix = cosine_similarity_chunked(df_all_articles_embeddings.tail(num_of_rows).values, chunk_size=chunk_size)

In [47]:
# # Compute cosine similarity matrix
# cosine_similarity_matrix = cosine_similarity(embeddings_df.tail(num_of_rows).values)

In [26]:
# print(cosine_similarity_matrix.shape)
# cosine_similarity_df = pd.DataFrame(cosine_similarity_matrix, index=embeddings_df.tail(num_of_rows).index, columns=embeddings_df.tail(num_of_rows).index)

In [9]:
# # Save the cosine similarity matrix to pickle file cause is very big
# cosine_similarity_df_file_path = f'./files/pickle/cosine_similarity_tail_{str(num_of_rows)}_{mult_way}.pkl'

# cosine_similarity_df.to_pickle(cosine_similarity_df_file_path)

In [18]:
# # Read cosine similarity matrix pickle file

# cosine_similarity_df_file_path = f'./files/pickle/cosine_similarity_tail_{str(num_of_rows)}_{mult_way}.pkl'
# cosine_similarity_df = pd.read_pickle(cosine_similarity_df_file_path)